In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import sys  
import tifffile as tf
from tqdm import tqdm
import os
import cv2

In [2]:
module_path = r"C:\Users\alexc\Documents\GitHub\Wide-Brain"
current_path = os.getcwd()
sys.path.insert(1, module_path)

In [3]:
from WFmovie import WFmovie
from WFmovie import create_channel
from WFmovie import ioi_epsilon_pathlength

In [4]:
def identify_folders(path, keywords):
    initial_folders = [f.path for f in os.scandir(path) if f.is_dir()]
    folders = []
    for folder in initial_folders:
        if all(keyword in folder for keyword in keywords):
            folders.append(folder)
    for i in range(len(folders)):
        folders[i] += '/'
    return folders

In [5]:
def identify_files(path, keywords):
    items = os.listdir(path)
    files = []
    for item in items:
        if all(keyword in item for keyword in keywords):
            files.append(item)
    files.sort()
    return files

In [6]:
def regress_timeseries(y, x, intercept=False, offset=False):
    """ Regress out the linear contribution of a regressor on time series data
    Args:
        y (ndarray): Time series specified as Nx1 vector. For M time series, specify as NxM array. 
        x (1D vector): NX1 time series to regress out of data.
        intercept (default = True) Specify wether or not to include an intercept term in the regressor.
        offset (default = False) Option to re-offset the regressed data at the estimated intercept
    Returns:
        eps (ndarray): Regressed data
    """
    if np.ndim(y)==1: #Make sure time series are presented as NX1 vectors
        y = y[:,None]
    N = np.shape(y)[0]
    if np.shape(x)[0] != N:
        raise Exception('Regressor must be of height N, with N the number of time points.')
    if np.ndim(x)==1: 
        x = x[:,None]
    if (offset and intercept) is False:
        intercept = True
    if intercept:
        x = np.insert(x,0,np.ones((1,N)),axis = 1)
    x_inv = np.linalg.pinv(x)
    beta = np.matmul(x_inv,y)
    y_est = np.matmul(x,beta)
    eps = y - y_est + np.mean(y)
    if offset:
        eps = eps+np.matmul(x[:,0][:,None],beta[0,:][None,:])
    return eps

In [7]:
def bin_pixels(frame, bin_size):
    height, width = frame.shape[:2]
    binned_height = height // bin_size
    binned_width = width // bin_size

    reshaped_frame = frame[:binned_height * bin_size, :binned_width * bin_size].reshape(binned_height, bin_size, binned_width, bin_size)
    binned_frame = np.sum(reshaped_frame, axis=(1, 3), dtype=np.float32)
    binned_frame = binned_frame / (bin_size**2)

    return binned_frame

In [8]:
def convert_to_hb(path_green, path_red, output_path):
    with tf.TiffFile(path_green) as tifG, tf.TiffFile(path_red) as tifR:
        R_green = tifG.pages
        R_red = tifR.pages
    
        num_frames = len(R_green) # Assuming the frames are along the first dimension
        frame_shape = R_green[0].asarray().shape
        stack_shape = (num_frames, frame_shape[0], frame_shape[1])

        lambda1 = 450 #nm
        lamba2 = 700 #nm
        npoints = 1000
        baseline_hbt = 100 #uM
        baseline_hbo = 60 #uM
        baseline_hbr = 40 #uM
        rescaling_factor = 1e6
        
        os.chdir(r"C:\Users\alexc\Documents\GitHub\Wide-Brain")
        eps_pathlength = ioi_epsilon_pathlength(lambda1, lamba2, npoints, baseline_hbt, baseline_hbo, baseline_hbr, filter=None)
        Ainv = np.linalg.pinv(eps_pathlength)*rescaling_factor
        os.chdir(current_path)

        with tf.TiffWriter(output_path+"\dHbO.tif", bigtiff=True) as writerHbO, tf.TiffWriter(output_path+"\dHbR.tif", bigtiff=True) as writerHbR, tf.TiffWriter(output_path+"\dHbT.tif", bigtiff=True) as writerHbT:
            for i in range(num_frames):
                data_G = R_green[i].asarray()
                data_R = R_red[i].asarray()
                ln_green = -np.log(data_G.flatten())
                ln_red = -np.log(data_R.flatten())
                ln_R = np.concatenate((ln_green.reshape(1, len(ln_green)), ln_red.reshape(1, len(ln_green))))
                Hbs = np.matmul(Ainv, ln_R)
                d_HbO = Hbs[0].reshape(frame_shape)
                d_HbR = Hbs[1].reshape(frame_shape)
                np.nan_to_num(d_HbO, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
                np.nan_to_num(d_HbR, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
                d_HbT = d_HbO + d_HbR
                
                writerHbO.write(np.float32(d_HbO), contiguous=True)
                writerHbR.write(np.float32(d_HbR), contiguous=True)
                writerHbT.write(np.float32(d_HbT), contiguous=True)

    return None

In [9]:
def speckle_contrast(data, kernel_size=7):
    kernel = np.ones((kernel_size, kernel_size)) / kernel_size**2
    result = np.empty(data.shape)
    for i, frame in enumerate(data):
        mean = ndi.correlate(frame, kernel, mode='nearest')
        std = np.sqrt(ndi.correlate(frame*frame, kernel, mode='nearest') - mean**2)
        K = std/mean
        result[i, :, :] = K[:, :]
    return result

## dHb pipeline

In [10]:
def dHb_pipeline(folder_path, save_path, baseline=[0, -1], bin_pixel=True, filter_data=True):
    # Create red channel and IOI_red.tif
    channel = 'red'
    path_red = save_path + r"\IOI_red.tif"
    create_channel(folder_path=folder_path, channel=channel)
    movie = WFmovie(folder_path=folder_path, channel=channel)

    data = movie.data / 4096
    data = np.flip(data, axis=(1,2))
    
    if filter_data:
        data = ndi.gaussian_filter1d(data, sigma=2.0, axis=0)
    
    norm_factor = np.mean(data[baseline[0]:baseline[1]], axis=0)
    if bin_pixel:
        norm_factor = bin_pixels(norm_factor, 2)
    
    with tf.TiffWriter(path_red, bigtiff=True) as writer:
        for frame in data:
            if bin_pixel:
                frame = bin_pixels(frame, 2)
            frame = frame / norm_factor
            writer.write(np.float32(frame), contiguous=True)
    data, movie = None, None
    
    # Create green channel and IOI_green.tif
    channel = 'green'
    path_green = save_path + r"\IOI_green.tif"
    create_channel(folder_path=folder_path, channel=channel)
    movie = WFmovie(folder_path=folder_path, channel=channel)

    data = movie.data / 4096
    data = np.flip(data, axis=(1,2))

    if filter_data:
        data = ndi.gaussian_filter1d(data, sigma=1.0, axis=0)
    
    norm_factor = np.mean(data[baseline[0]:baseline[1]], axis=0)
    if bin_pixel:
        norm_factor = bin_pixels(norm_factor, 2)
    
    with tf.TiffWriter(path_green, bigtiff=True) as writer:
        for frame in data:
            if bin_pixel:
                frame = bin_pixels(frame, 2)
            frame = frame / norm_factor
            writer.write(np.float32(frame), contiguous=True)
    data, movie = None, None
    
    # Create dHbO.tif, dHbR.tif and dHbT.tif
    convert_to_hb(path_green, path_red, save_path)
        

In [11]:
# Single file setup
setup = False
if setup:
    day = "STest"
    mouse = "RS_M32_10aout"
    path = r"C:\Users\alexc\OneDrive\Bureau\Doctorat_Neuro\CVR_dataset\{}\Dataset\{}".format(day, mouse)
    dHb_pipeline(folder_path=path, save_path=path, baseline=[0, -1], bin_pixel=True, filter_data=False)

KeyboardInterrupt: 

In [15]:
# Multiple files setup
setup = False
if setup:
    general_path = r"C:\Users\alexc\OneDrive\Bureau\Doctorat_Neuro\CVR_dataset\S4\Dataset"
    keyword = ["RS_M3"]
    folders = identify_folders(general_path, keyword)
    for path in tqdm(folders):
        print(path)
        dHb_pipeline(folder_path=path, save_path=path, baseline=[0, -1], bin_pixel=True, filter_data=True)

## LSCI pipeline

In [45]:
def LSCI_pipeline(folder_path, save_path, background_path=None, baseline=[0, -1], filter_data=True):
    channel = "ir"   
    create_channel(folder_path=folder_path, channel=channel)
    movie = WFmovie(folder_path=folder_path, channel=channel)
    data = movie.data / 4096
    
    if background_path is not None:
        create_channel(folder_path=background_path, channel=channel)
        movie = WFmovie(folder_path=background_path, channel=channel)
        background = np.mean(movie.data / 4096, axis=0)
        data = data - background
        
    static = np.mean(data, axis=(1,2))
    T, N, M = data.shape
    time_series = data.transpose(1, 2, 0).reshape(N * M, T)
    result = [regress_timeseries(series, static) for series in time_series]
    data = np.array(result).reshape(N, M, T).transpose(2, 0, 1)
    time_series, result = None, None
    data = speckle_contrast(data)
    data = 1 / np.power(data, 2)
    
    if baseline is not None:
        norm_factor = np.mean(data[baseline[0]:baseline[1]], axis=0)
        data = data / norm_factor
    
    if filter_data:
        data = ndi.gaussian_filter1d(data, sigma=2.0, axis=0)
    
    tf.imwrite(save_path + r"\LSCI.tif", np.float32(data))
    data = None
    
    return None

In [46]:
# Single file setup
setup = True
if setup:
    day = "STest"
    mouse = "RS_M38_22aout"
    path = r"C:\Users\alexc\OneDrive\Bureau\Doctorat_Neuro\CVR_dataset\{}\Dataset\{}".format(day, mouse)
    LSCI_pipeline(folder_path=path, save_path=path, background_path=None, baseline=None, filter_data=False)

In [18]:
# Multiple files setup
setup = False
if setup:
    general_path = r"C:\Users\alexc\OneDrive\Bureau\Doctorat_Neuro\CVR_dataset\S4\Dataset"
    keyword = ["RS_M3"]
    folders = identify_folders(general_path, keyword)
    for path in tqdm(folders):
        print(path)
        dHb_pipeline(folder_path=path, save_path=path, baseline=[0, -1], bin_pixel=True, filter_data=True)

## Monitoring pipeline

In [12]:
def convert_to_grayscale(input_file, output_file, frame_range):
    video = cv2.VideoCapture(input_file)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    output_file_template = output_file.replace('.tif', '_{}.tif')

    current_frame = frame_range[0]
    file_counter = 0
    max_frame = min(12000, frame_range[1]-frame_range[0])
    while current_frame < frame_range[1]:
        output_file = output_file_template.format(file_counter)
        print(output_file)
        with tf.TiffWriter(output_file, bigtiff=True) as tiff_writer:
            video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
            for _ in tqdm(range(max_frame)):
                ret, frame = video.read()

                # frame = frame[:, 420:1500]
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame = np.uint8(bin_pixels(frame, 2))
                tiff_writer.write(frame, contiguous=True)
                
                current_frame += 1
                if current_frame > frame_range[1]:
                    break
                    
            file_counter += 1
            
    frame = None
    video.release()
    
    return None


def write_frame_difference(input_folder, input_file, output_file, frame_range, binning=False):
    video = cv2.VideoCapture(f"{input_folder}\\{input_file}")
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_range[1] == -1:
        frame_range[1] = total_frames - 1

    motion_energy = []
    frames_to_process = frame_range[1] - frame_range[0] + 1
    num_output_files = (frames_to_process + 11999) // 12000
    for file_counter in range(num_output_files):
        start_frame = frame_range[0] + file_counter * 12000
        end_frame = min(frame_range[0] + (file_counter + 1) * 12000 - 1, frame_range[1])
        num_frames_in_output = end_frame - start_frame + 1
        output_file_template = output_file.replace('.tif', f'_{file_counter}.tif')
        print(output_file_template)

        with tf.TiffWriter(output_file_template, bigtiff=True) as tiff_writer:
            video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            ret, frame1 = video.read()
            frame1 = cv2.cvtColor(frame1[:, 420:1500], cv2.COLOR_BGR2GRAY)
            for current_frame in tqdm(range(start_frame, end_frame)):
                ret, frame2 = video.read()
                frame2 = cv2.cvtColor(frame2[:, 420:1500], cv2.COLOR_BGR2GRAY)
                
                diff_frame = cv2.absdiff(frame2, frame1)
                if binning:
                    diff_frame = bin_pixels(diff_frame, 2)
                tiff_writer.write(np.uint8(diff_frame), contiguous=True)
                motion_energy.append(np.mean(diff_frame))
                
                frame1 = frame2

    video.release()
    np.save(f"{input_folder}\\motion_energy", np.array(motion_energy))   
    
    return None


def remove_artefacts(input_folder, threshold):
    motion_energy_path = os.path.join(input_folder, 'motion_energy.npy')
    if os.path.exists(motion_energy_path):
        motion_energy = np.load(motion_energy_path)

        for idx, value in enumerate(motion_energy):
            if value < threshold:
                if idx == 0:
                    motion_energy[idx] = (value + motion_energy[idx + 1]) / 2
                elif idx == len(motion_energy) - 1:
                    motion_energy[idx] = (motion_energy[idx - 1] + value) / 2
                else:
                    motion_energy[idx] = (motion_energy[idx - 1] + motion_energy[idx + 1]) / 2

        np.save(motion_energy_path, motion_energy)
        motion_energy = None
    else:
        print(f"No 'motion_energy.npy' found in {input_folder}")

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.tif'):
            input_file = os.path.join(input_folder, file_name)

            with tf.TiffFile(input_file) as tif:
                frames = tif.asarray()
            
            for idx, frame in enumerate(frames):
                frame_mean = np.mean(frame)
                
                if frame_mean < threshold:
                    if idx == 0:
                        frames[idx] = (frame + frames[idx + 1]) // 2
                    elif idx == len(frames) - 1:
                        frames[idx] = (frames[idx - 1] + frame) // 2
                    else:
                        frames[idx] = (frames[idx - 1] + frames[idx + 1]) // 2
            
            tf.imwrite(input_file, frames)
            frames = None
            
    return None

In [15]:
setup = True
if setup:
    day = "STest"
    dataset = r"RS_M38_22aout"
    start = 11
    frame_range = [start, start+5400]
    folder_path = r"C:\Users\alexc\OneDrive\Bureau\Doctorat_Neuro\CVR_dataset\{}\Monitoring\{}_video".format(day, dataset)
    output_file = folder_path + r"\{}_monitoring.tif".format(dataset)

    write_frame_difference(folder_path, r"\{}_video.mp4".format(dataset), output_file, frame_range, binning=True)
    remove_artefacts(folder_path, 0.5)
#     convert_to_grayscale(folder_path+r"\{}_video.mp4".format(dataset), output_file, frame_range)

C:\Users\alexc\OneDrive\Bureau\Doctorat_Neuro\CVR_dataset\STest\Monitoring\RS_M38_22aout_video\RS_M38_22aout_monitoring_0.tif


100%|██████████| 5400/5400 [01:27<00:00, 61.63it/s]
